<a href="https://colab.research.google.com/github/minhquan1910/Movie_review_cnn/blob/main/Movie_review_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!tar -xf /content/review_polarity.tar.gz

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
import re
from os import listdir
from nltk.corpus import stopwords
from collections import Counter
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [ ]:
def load_doc(filename):
  file = open(filename, 'r')
  text = file.read()
  file.close()
  return text
def clean_doc(doc):
  text = doc.split()
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  tokens = [re_punc.sub('',w) for w in text]
  tokens = [token for token in tokens if token.isalpha()]
  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if not token in stop_words]
  tokens = [token for token in tokens if len(token) >1]
  return tokens

In [ ]:
filename ='/content/txt_sentoken/pos/cv000_29590.txt'
doc = load_doc(filename)
tokens = clean_doc(doc)
print(tokens)

['films', 'adapted', 'comic', 'books', 'plenty', 'success', 'whether', 'theyre', 'superheroes', 'batman', 'superman', 'spawn', 'geared', 'toward', 'kids', 'casper', 'arthouse', 'crowd', 'ghost', 'world', 'theres', 'never', 'really', 'comic', 'book', 'like', 'hell', 'starters', 'created', 'alan', 'moore', 'eddie', 'campbell', 'brought', 'medium', 'whole', 'new', 'level', 'mid', 'series', 'called', 'watchmen', 'say', 'moore', 'campbell', 'thoroughly', 'researched', 'subject', 'jack', 'ripper', 'would', 'like', 'saying', 'michael', 'jackson', 'starting', 'look', 'little', 'odd', 'book', 'graphic', 'novel', 'pages', 'long', 'includes', 'nearly', 'consist', 'nothing', 'footnotes', 'words', 'dont', 'dismiss', 'film', 'source', 'get', 'past', 'whole', 'comic', 'book', 'thing', 'might', 'find', 'another', 'stumbling', 'block', 'hells', 'directors', 'albert', 'allen', 'hughes', 'getting', 'hughes', 'brothers', 'direct', 'seems', 'almost', 'ludicrous', 'casting', 'carrot', 'top', 'well', 'anythi

In [ ]:
def add_doc_to_vocab(filename,vocab):
  doc = load_doc(filename)
  tokens = clean_doc(doc)
  vocab.update(tokens)
def process_docs(directory,vocab):
  for filename in listdir(directory):
    if filename.startswith('cv9'):
      continue
    path = directory + '/' + filename
    add_doc_to_vocab(path,vocab)

In [ ]:
vocab = Counter()
process_docs('/content/txt_sentoken/neg',vocab)
process_docs('/content/txt_sentoken/pos',vocab)
print(len(vocab))
print(vocab.most_common(50))
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))
def save_list(lines,filename):
  doc = '\n'.join(lines)
  file = open(filename, 'w')
  file.write(doc)
  file.close()
save_list(tokens,'vocab.txt')

44276
[('film', 7983), ('one', 4946), ('movie', 4826), ('like', 3201), ('even', 2262), ('good', 2080), ('time', 2041), ('story', 1907), ('films', 1873), ('would', 1844), ('much', 1824), ('also', 1757), ('characters', 1735), ('get', 1724), ('character', 1703), ('two', 1643), ('first', 1588), ('see', 1557), ('way', 1515), ('well', 1511), ('make', 1418), ('really', 1407), ('little', 1351), ('life', 1334), ('plot', 1288), ('people', 1269), ('bad', 1248), ('could', 1248), ('scene', 1241), ('movies', 1238), ('never', 1201), ('best', 1179), ('new', 1140), ('scenes', 1135), ('man', 1131), ('many', 1130), ('doesnt', 1118), ('know', 1092), ('dont', 1086), ('hes', 1024), ('great', 1014), ('another', 992), ('action', 985), ('love', 977), ('us', 967), ('go', 952), ('director', 948), ('end', 946), ('something', 945), ('still', 936)]
25767


In [ ]:
def clean_doc(doc,vocab):
  tokens = doc.split()
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  tokens = [re_punc.sub('',w) for w in tokens]
  tokens = [w for w in tokens if w in vocab]
  tokens = ' '.join(tokens)
  return tokens
def process_docs(directory,vocab,is_train):
  document = list()
  for filename in listdir(directory):
    if is_train and filename.startswith('cv9'):
      continue
    if not is_train and not filename.startswith('cv9'):
      continue
    path = directory + '/' + filename
    doc = load_doc(path)
    tokens = clean_doc(doc,vocab)
    document.append(tokens)
  return document
def load_clean_dataset(vocab,is_train):
  neg = process_docs('/content/txt_sentoken/neg',vocab,is_train)
  pos = process_docs('/content/txt_sentoken/pos',vocab,is_train)
  docs = neg + pos
  labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
  return docs,labels
def create_tokenizer(doc):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(doc)
  return tokenizer
def encoded_docs(tokenizer,max_length,docs):
  encoded = tokenizer.texts_to_sequences(docs)
  padding = pad_sequences(encoded,maxlen=max_length,padding = 'post')
  return padding
def define_model(vocab_size,max_length):
  model = Sequential()
  model.add(Embedding(vocab_size,100,input_length = max_length))
  model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(10,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = 'accuracy')
  model.summary()
  plot_model(model, to_file = 'model.png', show_shapes = True)
  return model 

In [ ]:
vocab_filename = '/content/vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())
train_docs, ytrain = load_clean_dataset(vocab,True)
tokenizer = create_tokenizer(train_docs)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size %d' %vocab_size)
max_length = max([len(s.split()) for s in train_docs])
print('Max length %d' %max_length)
Xtrain = encoded_docs(tokenizer,max_length,train_docs)
model = define_model(vocab_size,max_length)
model.fit(Xtrain, ytrain, epochs = 10, verbose = 2)
model.save('model.h5')

Vocabulary size 25768
Max length 1317
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1317, 100)         2576800   
_________________________________________________________________
conv1d (Conv1D)              (None, 1310, 32)          25632     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 655, 32)           0         
_________________________________________________________________
flatten (Flatten)            (None, 20960)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                209610    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 2,812,053
Trainable params: 2,812,053
Non-trainable params: 0
________

In [ ]:
from keras.models import load_model
test_docs,ytest = load_clean_dataset(vocab,False)
Xtest = encoded_docs(tokenizer,max_length,test_docs)
model = load_model('model.h5')
_, acc = model.evaluate(Xtrain,ytrain,verbose = 0)
print('Train accuracy %.2f' %(acc*100))
_, acc = model.evaluate(Xtest,ytest,verbose=0)
print('Test accuracy %.2f' %(acc*100))


Train accuracy 100.00
Test accuracy 86.50


In [ ]:
def predict_sentiment(review,vocab,tokenizer,max_length,model):
  line = clean_doc(review,vocab)
  padded = encoded_docs(tokenizer,max_length,[line])
  yhat = model.predict(padded,verbose = 0)
  percent_pos = yhat[0,0]
  if round(percent_pos) == 0:
    return (1-percent_pos), 'NEGATIVE'
  return (1-percent_pos), 'POSITIVE'
text = 'Everyone wil enjoy this film.I love it, recommended!'
percent,sentiment = predict_sentiment(text,vocab,tokenizer,max_length,model)
print('Review: [%s] \nSentiment: %s (%.3f%%)' %(text,sentiment,percent*100))
text = 'This is a bad movie.Do not watch it.It sucks'
percent,sentiment = predict_sentiment(text,vocab,tokenizer,max_length,model)
print('Review: [%s] \nSentiment: %s (%.3f%%)' %(text,sentiment,percent*100))

Review: [Everyone wil enjoy this film.I love it, recommended!] 
Sentiment: POSITIVE (24.511%)
Review: [This is a bad movie.Do not watch it.It sucks] 
Sentiment: POSITIVE (29.346%)
